In [299]:
import pyspark
from pyspark.sql.types import *
import os
import boto3
import numpy as np
from pyspark.sql.functions import *

In [2]:
sc = pyspark.SparkContext.getOrCreate()
ss = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
bucket_name = 'msds697jonross.and.friends' # Add your bucket name
file_name = 'sfpd.csv' # select file
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name) 
obj = bucket.Object(key=file_name) # S3 uses key-value structure where key is your file name
file_content = obj.get()["Body"].read().decode("utf-8") # Read the Body which is the contents of the file.

In [4]:
# number of rows (subract header and empty line at end)
rows = file_content.split('\n')
len(rows)-2

2170785

In [63]:
rows[:10]

['unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp',
 '166018573,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,,NONE,100 Block of VELASCO AV,-122.4133519852842,37.70820245849022,"(37.70820245849022, -122.4133519852842)",16601857306244,2016-01-17 23:54:00+00:00',
 '160874408,DRIVING UNDER THE INFLUENCE,DRIVING WHILE UNDER THE INFLUENCE OF DRUGS,Thursday,PARK,NONE,DUBOCE AV / NOE ST,-122.43357509728241,37.769176747627725,"(37.769176747627725, -122.43357509728241)",16087440865060,2016-10-27 04:15:00+00:00',
 '120574836,ROBBERY,ATTEMPTED ROBBERY ON THE STREET WITH A KNIFE,Saturday,PARK,NONE,HAIGHT ST / DIVISADERO ST,-122.437048523435,37.7712678186367,"(37.7712678186367, -122.437048523435)",12057483603412,2012-07-21 05:55:00+00:00',
 '160573898,SECONDARY CODES,JUVENILE INVOLVED,Saturday,PARK,NONE,1200 Block of PAGE ST,-122.44139370111428,37.77172731417711,"(37.77172731417711, -122.44139370111428)",16057389815500,2016-07-16 1

In [47]:
# number of  columns
column_names = rows[0]
len(column_names.split(','))

12

In [48]:
print('   '.join(x for x in column_names.split(',')))

unique_key   category   descript   dayofweek   pddistrict   resolution   address   longitude   latitude   location   pdid   timestamp


In [49]:
# randomly sample rows
sz=10000
samples = np.random.choice(rows[1:], size=sz, replace=False)
samples[:2]

array(['61110088,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Tuesday,CENTRAL,NONE,STOCKTON ST / SACRAMENTO ST,-122.407729769991,37.7931773025837,"(37.7931773025837, -122.407729769991)",6111008806244,2006-10-17 17:20:00+00:00',
       '30891346,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,MISSION,"ARREST, CITED",16TH ST / VALENCIA ST,-122.421886357989,37.7649178909469,"(37.7649178909469, -122.421886357989)",3089134665016,2003-07-28 02:33:00+00:00'],
      dtype='<U286')

In [62]:
for i in range(100):
    print(samples[i].split(","))

['61110088', 'LARCENY/THEFT', 'GRAND THEFT FROM LOCKED AUTO', 'Tuesday', 'CENTRAL', 'NONE', 'STOCKTON ST / SACRAMENTO ST', '-122.407729769991', '37.7931773025837', '"(37.7931773025837', ' -122.407729769991)"', '6111008806244', '2006-10-17 17:20:00+00:00']
['30891346', 'OTHER OFFENSES', '"DRIVERS LICENSE', ' SUSPENDED OR REVOKED"', 'Monday', 'MISSION', '"ARREST', ' CITED"', '16TH ST / VALENCIA ST', '-122.421886357989', '37.7649178909469', '"(37.7649178909469', ' -122.421886357989)"', '3089134665016', '2003-07-28 02:33:00+00:00']
['101145539', 'FRAUD', '"FALSE PRETENSES', ' PETTY THEFT"', 'Friday', 'TARAVAL', 'NONE', '100 Block of FONT BL', '-122.473644075533', '37.7155536666978', '"(37.7155536666978', ' -122.473644075533)"', '10114553910120', '2010-12-10 12:00:00+00:00']
['110712024', 'OTHER OFFENSES', 'CONSPIRACY', 'Monday', 'NORTHERN', '"ARREST', ' BOOKED"', 'SACRAMENTO ST / POLK ST', '-122.420874632415', '37.7914943051906', '"(37.7914943051906', ' -122.420874632415)"', '1107120242608

In [250]:
def map_police(x):
    my_str = ''
    stg1 = x.split(',"')
    part_1 = stg1[0]
    my_str = my_str + part_1
    stg2 = [x.split('",') for x in stg1[1:]]
    for i in range(len(stg2)):
        new = stg2[i][0].replace(',',';')
        my_str = my_str + ',' + new
        for j in range(len(stg2[i])-1):
            new = new = stg2[i][j+1]
            my_str = my_str + ',' + new
    return my_str.split(',')


def float_safe_police(row):
    try:
        row[7] = float(row[7])
    except ValueError:
        row[7] =  None
    try:
        row[8] = float(row[8])
    except ValueError:
        row[8] =  None
    return row

In [251]:
sc.parallelize(list(samples))\
    .map(map_police)\
    .map(float_safe_police).take(10)

[['61110088',
  'LARCENY/THEFT',
  'GRAND THEFT FROM LOCKED AUTO',
  'Tuesday',
  'CENTRAL',
  'NONE',
  'STOCKTON ST / SACRAMENTO ST',
  -122.407729769991,
  37.7931773025837,
  '(37.7931773025837; -122.407729769991)',
  '6111008806244',
  '2006-10-17 17:20:00+00:00'],
 ['30891346',
  'OTHER OFFENSES',
  'DRIVERS LICENSE; SUSPENDED OR REVOKED',
  'Monday',
  'MISSION',
  'ARREST; CITED',
  '16TH ST / VALENCIA ST',
  -122.421886357989,
  37.7649178909469,
  '(37.7649178909469; -122.421886357989)',
  '3089134665016',
  '2003-07-28 02:33:00+00:00'],
 ['101145539',
  'FRAUD',
  'FALSE PRETENSES; PETTY THEFT',
  'Friday',
  'TARAVAL',
  'NONE',
  '100 Block of FONT BL',
  -122.473644075533,
  37.7155536666978,
  '(37.7155536666978; -122.473644075533)',
  '10114553910120',
  '2010-12-10 12:00:00+00:00'],
 ['110712024',
  'OTHER OFFENSES',
  'CONSPIRACY',
  'Monday',
  'NORTHERN',
  'ARREST; BOOKED',
  'SACRAMENTO ST / POLK ST',
  -122.420874632415,
  37.7914943051906,
  '(37.7914943051906; 

In [252]:
rdd = sc.parallelize(list(samples))\
    .map(map_police)\
    .map(float_safe_police)\
    .filter(lambda x: len(x)==len(column_names.split(',')))

In [254]:
sc.parallelize(list(samples))\
    .filter(lambda x: x.split(',')[0]=='101145539')\
    .map(map_police)\
    .map(float_safe_police)\
    .collect()

[['101145539',
  'FRAUD',
  'FALSE PRETENSES; PETTY THEFT',
  'Friday',
  'TARAVAL',
  'NONE',
  '100 Block of FONT BL',
  -122.473644075533,
  37.7155536666978,
  '(37.7155536666978; -122.473644075533)',
  '10114553910120',
  '2010-12-10 12:00:00+00:00']]

In [255]:
# number of rows removed
sz - rdd.count()

0

In [256]:
'unique_key,category,descript,dayofweek,pddistrict,resolution,address,longitude,latitude,location,pdid,timestamp'
schema = StructType([StructField("unique_key", StringType(), False),
                    StructField("category", StringType(), False),
                    StructField("descript", StringType(), False),
                    StructField("dayofweek", StringType(), False),
                    StructField("pddistrict", StringType(), False),
                    StructField("resolution", StringType(), False),
                    StructField("address", StringType(), False),
                    StructField("longitude", FloatType(), False),
                    StructField("latitude", FloatType(), False),
                    StructField("location", StringType(), False),
                    StructField("pdid", StringType(), False),
                    StructField("timestamp", StringType(), False)
                    ])

In [329]:
df = ss.createDataFrame(rdd, schema)
df = df.withColumn('date', to_timestamp(df.timestamp, 'yyyy-MM-dd HH:mm:00+00:00'))\
    .drop("timestamp")\
    .withColumnRenamed('date', 'timestamp')

In [330]:
# print('\n----------------------\n'.join(x for x in column_names.split(',')))
df.printSchema()

root
 |-- unique_key: string (nullable = false)
 |-- category: string (nullable = false)
 |-- descript: string (nullable = false)
 |-- dayofweek: string (nullable = false)
 |-- pddistrict: string (nullable = false)
 |-- resolution: string (nullable = false)
 |-- address: string (nullable = false)
 |-- longitude: float (nullable = false)
 |-- latitude: float (nullable = false)
 |-- location: string (nullable = false)
 |-- pdid: string (nullable = false)
 |-- timestamp: timestamp (nullable = true)



In [331]:
df.groupBy("dayofweek")\
    .count()\
    .orderBy("count", ascending=[0])\
    .show()

+---------+-----+
|dayofweek|count|
+---------+-----+
|   Friday| 1529|
| Thursday| 1513|
| Saturday| 1456|
|  Tuesday| 1450|
|Wednesday| 1416|
|   Monday| 1407|
|   Sunday| 1229|
+---------+-----+



In [334]:
df.groupBy("pddistrict")\
    .count()\
    .orderBy("count", ascending=[0])\
    .show()

+----------+-----+
|pddistrict|count|
+----------+-----+
|  SOUTHERN| 1808|
|   MISSION| 1352|
|  NORTHERN| 1210|
|   BAYVIEW| 1015|
|   CENTRAL| 1015|
| INGLESIDE|  888|
|TENDERLOIN|  854|
|   TARAVAL|  751|
|      PARK|  572|
|  RICHMOND|  535|
+----------+-----+



In [335]:
df.groupBy("category")\
    .count()\
    .orderBy("count", ascending=[0])\
    .show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|       LARCENY/THEFT| 2217|
|      OTHER OFFENSES| 1377|
|        NON-CRIMINAL| 1080|
|             ASSAULT|  864|
|       VEHICLE THEFT|  565|
|       DRUG/NARCOTIC|  537|
|            WARRANTS|  488|
|           VANDALISM|  485|
|            BURGLARY|  417|
|      SUSPICIOUS OCC|  367|
|      MISSING PERSON|  280|
|             ROBBERY|  256|
|               FRAUD|  184|
|FORGERY/COUNTERFE...|  136|
|     SECONDARY CODES|  116|
|         WEAPON LAWS|   88|
|        PROSTITUTION|   85|
|            TRESPASS|   84|
|         DRUNKENNESS|   45|
|     STOLEN PROPERTY|   42|
+--------------------+-----+
only showing top 20 rows



In [336]:
df.groupBy("resolution")\
    .count()\
    .orderBy("count", ascending=[0])\
    .show()

+--------------------+-----+
|          resolution|count|
+--------------------+-----+
|                NONE| 6208|
|      ARREST; BOOKED| 2408|
|       ARREST; CITED|  690|
|             LOCATED|  144|
|   PSYCHOPATHIC CASE|  127|
|           UNFOUNDED|  111|
|     JUVENILE BOOKED|   71|
|COMPLAINANT REFUS...|   50|
|      NOT PROSECUTED|   48|
|DISTRICT ATTORNEY...|   40|
|      JUVENILE CITED|   29|
|EXCEPTIONAL CLEAR...|   25|
|PROSECUTED BY OUT...|   23|
| JUVENILE ADMONISHED|   16|
|   JUVENILE DIVERTED|    6|
|CLEARED-CONTACT J...|    3|
|PROSECUTED FOR LE...|    1|
+--------------------+-----+



In [337]:
df.select("timestamp").show(1, False)

+-------------------+
|timestamp          |
+-------------------+
|2006-10-17 17:20:00|
+-------------------+
only showing top 1 row



In [359]:
def t_series(interval='day'):
    df.select(date_trunc(interval, 'timestamp'))\
        .withColumnRenamed(f'date_trunc({interval}, timestamp)', interval)\
        .groupBy(interval)\
        .count()\
        .orderBy(interval, ascending=False).show()

In [361]:
t_series('year')

+-------------------+-----+
|               year|count|
+-------------------+-----+
|2018-01-01 00:00:00|   20|
|2017-01-01 00:00:00|  730|
|2016-01-01 00:00:00|  684|
|2015-01-01 00:00:00|  676|
|2014-01-01 00:00:00|  684|
|2013-01-01 00:00:00|  706|
|2012-01-01 00:00:00|  627|
|2011-01-01 00:00:00|  606|
|2010-01-01 00:00:00|  614|
|2009-01-01 00:00:00|  638|
|2008-01-01 00:00:00|  651|
|2007-01-01 00:00:00|  651|
|2006-01-01 00:00:00|  645|
|2005-01-01 00:00:00|  673|
|2004-01-01 00:00:00|  693|
|2003-01-01 00:00:00|  702|
+-------------------+-----+

